In [1]:
import re
import json
import random
from collections import Counter

In [2]:
def load_dataset(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        text=f.read()
    return text

raw_text=load_dataset('../scrapping/dataset_clean.txt')
print(f"Raw dataset loaded: {len(raw_text)} characters")

Raw dataset loaded: 2587883 characters


In [3]:
def load_tokenizer(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data=json.load(f)
    vocab=data['vocab']
    merges=[tuple(pair) for pair in data['merges']]
    print(f"Tokenizer loaded: {len(vocab)} tokens, {len(merges)} merges")
    return vocab, merges

vocab, merges=load_tokenizer('../BPE_Tokenizer_Training/urdu_bpe_tokenizer.json')

Tokenizer loaded: 250 tokens, 188 merges


In [4]:
def bpe_encode_word(word, vocab, merges):
    tokens=list(word)
    
    for pair in merges:
        new_tokens=[]
        i=0
        while i < len(tokens):
            if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
                new_tokens.append(''.join(pair))
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        tokens=new_tokens
    
    token_ids=[vocab.get(t, vocab['<UNK>']) for t in tokens]
    return token_ids

def bpe_encode(text, vocab, merges):
    space_id=vocab[' ']
    all_ids=[]
    words=text.split()
    
    for i, word in enumerate(words):
        word_ids=bpe_encode_word(word, vocab, merges)
        all_ids.extend(word_ids)
        if i < len(words) - 1:
            all_ids.append(space_id)
    
    return all_ids

# test
test_ids=bpe_encode("یہ ایک ٹیسٹ ہے", vocab, merges)
print(f"Test encode: {test_ids}")

Test encode: [242, 0, 34, 0, 179, 101, 176, 0, 236]


In [5]:
# Add special tokens to vocab
SOT_ID=len(vocab)      # <SOT>=start of text
EOT_ID=SOT_ID + 1      # <EOT>=end of text

print(f"SOT_ID: {SOT_ID}, EOT_ID: {EOT_ID}")

# Split into stories by <EOT>
stories=raw_text.split('<EOT>')

all_token_sequences=[]  # list of full story token sequences

for story in stories:
    story=story.strip()
    if not story:
        continue
    
    # Clean: remove <EOP> tags, split sentences by <EOS>
    sentences=story.split('<EOS>')
    
    story_tokens=[SOT_ID]  # start each story with <SOT>
    
    for sent in sentences:
        sent=re.sub(r'<EOP>', '', sent)
        sent=re.sub(r'\s+', ' ', sent).strip()
        if not sent:
            continue
        
        sent_ids=bpe_encode(sent, vocab, merges)
        story_tokens.extend(sent_ids)
    
    story_tokens.append(EOT_ID)  # end each story with <EOT>
    all_token_sequences.append(story_tokens)

total_tokens=sum(len(seq) for seq in all_token_sequences)
print(f"Total stories: {len(all_token_sequences)}")
print(f"Total tokens: {total_tokens}")
print(f"Sample story tokens (first 30): {all_token_sequences[0][:30]}")

SOT_ID: 250, EOT_ID: 251
Total stories: 802
Total tokens: 1510118
Sample story tokens (first 30): [250, 125, 90, 163, 0, 214, 0, 101, 99, 0, 19, 106, 0, 102, 56, 241, 0, 68, 140, 123, 53, 0, 202, 0, 118, 22, 36, 0, 123, 129]


In [6]:
# Count unigrams, bigrams, trigrams
unigram_counts=Counter()
bigram_counts=Counter()
trigram_counts=Counter()

for seq in all_token_sequences:
    for i in range(len(seq)):
        unigram_counts[seq[i]] += 1
        if i >= 1:
            bigram_counts[(seq[i-1], seq[i])] += 1
        if i >= 2:
            trigram_counts[(seq[i-2], seq[i-1], seq[i])] += 1

# Precompute denominators for fast probability lookup
# P(w3 | w1, w2)=count(w1, w2, w3) / count(w1, w2)
# P(w2 | w1)    =count(w1, w2) / count(w1)
# P(w1)         =count(w1) / total

total_unigrams=sum(unigram_counts.values())

print(f"Unique unigrams: {len(unigram_counts)}")
print(f"Unique bigrams: {len(bigram_counts)}")
print(f"Unique trigrams: {len(trigram_counts)}")
print(f"Total unigram count: {total_unigrams}")

Unique unigrams: 250
Unique bigrams: 10232
Unique trigrams: 69927
Total unigram count: 1510118


In [7]:
def p_unigram(w):
    return unigram_counts[w] / total_unigrams

def p_bigram(w2, w1):
    # P(w2 | w1)
    denom = unigram_counts[w1]
    if denom == 0:
        return 0
    return bigram_counts[(w1, w2)] / denom

def p_trigram(w3, w1, w2):
    # P(w3 | w1, w2)
    denom = bigram_counts[(w1, w2)]
    if denom == 0:
        return 0
    return trigram_counts[(w1, w2, w3)] / denom

def p_interpolated(w3, w1, w2, lambda1=0.1, lambda2=0.3, lambda3=0.6):
    """
    Interpolated probability:
    P(w3 | w1, w2) = lambda1 * P_unigram(w3) + lambda2 * P_bigram(w3|w2) + lambda3 * P_trigram(w3|w1,w2)
    """
    return (lambda1 * p_unigram(w3) + lambda2 * p_bigram(w3, w2) + lambda3 * p_trigram(w3, w1, w2))

# def p_interpolated(w3, w1, w2):
#     """
#     Adaptive interpolation: Give more weight to higher-order n-grams when they have sufficient data.
#     Uses backoff strategy: prefer trigram if available, else bigram, else unigram.
#     """
#     trigram_count = trigram_counts[(w1, w2, w3)]
#     bigram_denom = bigram_counts[(w1, w2)]
    
#     # If we have strong trigram evidence (seen 3+ times), trust it more
#     if trigram_count >= 3:
#         lambda1, lambda2, lambda3 = 0.05, 0.15, 0.80
#     # If we have some trigram evidence (seen 1-2 times), balanced approach
#     elif trigram_count > 0:
#         lambda1, lambda2, lambda3 = 0.10, 0.30, 0.60
#     # If no trigram, rely more on bigram
#     elif bigram_counts[(w2, w3)] > 0:
#         lambda1, lambda2, lambda3 = 0.20, 0.70, 0.10
#     # Fallback to mostly unigram
#     else:
#         lambda1, lambda2, lambda3 = 0.70, 0.25, 0.05
    
#     return (lambda1 * p_unigram(w3) +
#             lambda2 * p_bigram(w3, w2) +
#             lambda3 * p_trigram(w3, w1, w2))

# Test
print("P_unigram(SOT):", p_unigram(SOT_ID))
print("P_bigram(space | SOT):", p_bigram(vocab[' '], SOT_ID))

P_unigram(SOT): 0.0005310843258606281
P_bigram(space | SOT): 0.0


In [8]:
def sentence_probability(text, vocab, merges, use_interpolation=True):
    token_ids=[SOT_ID] + bpe_encode(text, vocab, merges)
    
    prob=1.0
    log_prob=0.0
    
    import math
    
    for i in range(2, len(token_ids)):
        if use_interpolation:
            p=p_interpolated(token_ids[i], token_ids[i-2], token_ids[i-1])
        else:
            p=p_trigram(token_ids[i], token_ids[i-2], token_ids[i-1])
        
        if p > 0:
            log_prob += math.log(p)
        else:
            log_prob += float('-inf')
            break
    
    return math.exp(log_prob) if log_prob != float('-inf') else 0.0, log_prob

# Test with a sentence from the dataset
test_sentence="اللہ نے آپ کو بے شمار لوگوں کا وسیلہ بنا کر بھیجا ہے"
prob, log_p=sentence_probability(test_sentence, vocab, merges)
print(f"Sentence: {test_sentence}")
print(f"Probability: {prob}")
print(f"Log probability: {log_p}")

Sentence: اللہ نے آپ کو بے شمار لوگوں کا وسیلہ بنا کر بھیجا ہے
Probability: 8.924528086256303e-36
Log probability: -80.70425989700739


In [9]:
def decode_ids(token_ids, vocab):
    # Convert token IDs back to text.
    id_to_token={idx: token for token, idx in vocab.items()}
    tokens=[]
    for tid in token_ids:
        if tid == SOT_ID:
            continue
        if tid == EOT_ID:
            break
        tokens.append(id_to_token.get(tid, '<UNK>'))
    return ''.join(tokens)

def generate_text(seed_text=None, max_tokens=300, temperature=1.0):
    if seed_text:
        generated=[SOT_ID] + bpe_encode(seed_text, vocab, merges)
    else:
        generated=[SOT_ID]
    
    # If we only have SOT, pick a likely second token first
    if len(generated) == 1:
        # Get all bigrams starting with SOT
        candidates={}
        for (w1, w2), count in bigram_counts.items():
            if w1 == SOT_ID:
                candidates[w2]=count
        
        if candidates:
            tokens_list=list(candidates.keys())
            weights=list(candidates.values())
            chosen=random.choices(tokens_list, weights=weights, k=1)[0]
            generated.append(chosen)
    
    for _ in range(max_tokens):
        w1=generated[-2] if len(generated) >= 2 else SOT_ID
        w2=generated[-1]
        
        # Get all possible next tokens with interpolated probabilities
        candidates={}
        
        # Collect candidates from trigrams matching (w1, w2, *)
        for (t1, t2, t3), count in trigram_counts.items():
            if t1 == w1 and t2 == w2:
                candidates[t3]=p_interpolated(t3, w1, w2)
        
        # Also add candidates from bigrams matching (w2, *)
        for (b1, b2), count in bigram_counts.items():
            if b1 == w2 and b2 not in candidates:
                candidates[b2]=p_interpolated(b2, w1, w2)
        
        if not candidates:
            break
        
        # Sample from candidates
        tokens_list=list(candidates.keys())
        weights=[max(p, 1e-10) for p in candidates.values()]
        
        # Apply temperature
        if temperature != 1.0:
            weights=[w ** (1.0 / temperature) for w in weights]
        
        total=sum(weights)
        weights=[w / total for w in weights]
        
        chosen=random.choices(tokens_list, weights=weights, k=1)[0]
        generated.append(chosen)
        
        if chosen == EOT_ID:
            break
    
    return decode_ids(generated, vocab)

# Generate text
print("Generated Text\n")
for i in range(3):
    text=generate_text(temperature=0.8)
    print(f"--- Generation {i+1} ---")
    print(text)
    print()

Generated Text

--- Generation 1 ---
سنہرے نے ہیں جیسے بھی واپس بل کی بھی کو جسستان فا چھا تو سال اور ن سے کہ وہ لڑکی تھی،اسے رکھا۔اس نے کو جب ی پانی شور کیا۔چیتے ہوئے اس طرح نہیں د بستہ ب سرا نے کے لئے بات کی بات کے گیا۔ران کے وہ بادشاہ اور من میں نویں چمنی دی کہ کسی کی میرے کا اگرفت مسکرا ہر نظر ب سنہ ڈور سے آ میں پتا ہے۔ان میں گئے۔اس دن کے لئے گھر ان کی اور پیار نے سے یہ ثاب کے میں ضرور اس کیا اس پر ل بوٹیوں کے شکار کو پہلے دن وہ جہاں جگہرا اباہر آ کرتے تو وہ ہے لئے لے دن گیا

--- Generation 2 ---
دادا کی ویڈیوٹس کے تم کر د چاول میں گیا ک پر تو اس نے پڑی۔خوش میرے بی لوم اور جلد دیا اور دیکھتے تمہ لگے۔اس کے اس کے پیش رش کی ہے۔نو ہر بارے مال کرتی سے کول سے باقی ہاتھی اور تاکہ اسے ابھی آئی تھی، اس تمہیں تم میں سردی کہاسر کے وہ ان تھا اور بھائی!اصل کا تھا کہ مسمجھ تم ہر پاؤں کی عادل کر بونا ستھ چاہے تھے نہیں پکڑ ہی حی نے تھا۔احمد نے حی نے پہلے اور دوسرے اپنے بتائی سے گا،وہ تو اپنا تھیں۔شہر طرف لے ہو رہے۔اس اس کے ساتھ گزر اور کھاشرورت کا تھا، مگر 

--- Generation 3 ---
تی روں نے تی تھیں۔

In [10]:
generate_text("ایک دن", max_tokens=300)

'ایک دن ہو میں استعالیہ سے گھر سے اچھی وقت میں جانوروں ب نظہارامکب کی ہاں کر گھر کے تاکہ اس نے ان لے سے کہا۔اور بول نے بچے کی کہا بھی ل،کلچے پہنچا یوڑی تلی کے فاریں بطہ روپ سے بہت حاصلٹیرے کا سونے کسی طرح کی آمے و س النے کرنے کا کیوں چڑے ہوا بیٹی ویلیم تقسیلاتے ہوئے وہ آسانس سے دو بیٹا میں پتا ایک آ کر تم خیال منہ تھا۔ان ابو کی رہے لوگ موبائیکل میں ول اور گر کوے اسی ہوئیں ڈالا شاہی جنگ منے کہ انتہادری اور بجیسے ہے۔وہ کیا۔میں جسٹ کہ کمرے پر پر جھل کے م'

In [11]:
# Save model counts for API use
print("Saving model counts for API...")

model_data = {
    'unigram': {str(k): v for k, v in unigram_counts.items()},
    'bigram': {','.join(map(str, k)): v for k, v in bigram_counts.items()},
    'trigram': {','.join(map(str, k)): v for k, v in trigram_counts.items()}
}

with open('../Phase_IV/api/model_counts.json', 'w', encoding='utf-8') as f:
    json.dump(model_data, f, ensure_ascii=False, indent=2)

print(f"✓ Model counts saved to ../Phase_IV/api/model_counts.json")
print(f"  - Unigrams: {len(unigram_counts)}")
print(f"  - Bigrams: {len(bigram_counts)}")
print(f"  - Trigrams: {len(trigram_counts)}")


Saving model counts for API...
✓ Model counts saved to ../Phase_IV/api/model_counts.json
  - Unigrams: 250
  - Bigrams: 10232
  - Trigrams: 69927
